In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#python setup_database.py & python setup_qdrant.py & python setup_elastic.py & python setup_redis.py

In [3]:
import sys
sys.path.append('/app')

In [4]:

from api_client import APIClient
from uuid import UUID
import asyncio
from datetime import datetime, timedelta

In [12]:

client = await APIClient.create(email="2280905@gmail.com")

In [11]:

client = await APIClient.create(email="dmitrygrankin0@gmail.com")

TypeError: cannot unpack non-iterable NoneType object

In [13]:
client

In [14]:

# Auth examples
#auth_result = await client.google_auth("google_token_here", {"utm_source": "web"})



In [15]:
#token_result = await client.submit_token("918c9f3899d54f3bbcf445cd053c2304")
    


### Contents

In [16]:
contents = await client.get_contents(
    only_archived=False,
    offset=0,
    limit=20
)

In [17]:
contents

{'total': 3,
 'contents': [{'content_id': '8a5f28e7-e970-44ba-9ced-740f176e69c3',
   'type': 'meeting',
   'text': None,
   'timestamp': '2025-02-04T12:30:11',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': [{'name': 'Dmitriy Grankin', 'type': 'speaker'}]},
  {'content_id': 'e36a3d84-a63e-4f86-b119-7bd4e7696c2b',
   'type': 'meeting',
   'text': None,
   'timestamp': '2025-02-04T11:15:46',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': [{'name': 'Artem Puzik', 'type': 'speaker'}]},
  {'content_id': '08aba88d-3c49-4969-8471-609666e7514a',
   'type': 'meeting',
   'text': None,
   'timestamp': '2025-02-04T10:02:37',
   'is_indexed': False,
   'is_owner': True,
   'access_level': 'owner',
   'entities': [{'name': 'Dmitriy Grankin', 'type': 'speaker'}]}]}

In [18]:
# Add new content
content = await client.add_content(
    body="Sample content text",
    content_type="note",
    entities=[{"type": "tag", "name": "test"}]
)


In [20]:
content_id = content["content_id"]

In [21]:
# Get single content
single_content = await client.get_content(content_id)
single_content



{'content_id': 'c7adad2c-d7f1-404b-bc4a-0d1e82bf0e75',
 'type': 'note',
 'text': 'Sample content text',
 'timestamp': '2025-02-04T14:58:43.128391',
 'is_indexed': False,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [{'name': 'test', 'type': 'tag'}],
 'parent_id': None,
 'meta': None}

In [22]:
archived = await client.archive_content(content_id)

In [23]:
archived

{'success': True}

In [24]:
content_id

'c7adad2c-d7f1-404b-bc4a-0d1e82bf0e75'

In [25]:
try:
    single_content = await client.get_content(content_id)
except Exception as e:
    print(e)


#OK - content unavailable after archive

Content not found or no access


In [26]:
restored = await client.restore_content(content_id)
single_content = await client.get_content(content_id)
single_content

{'content_id': 'c7adad2c-d7f1-404b-bc4a-0d1e82bf0e75',
 'type': 'note',
 'text': 'Sample content text',
 'timestamp': '2025-02-04T14:58:43.128391',
 'is_indexed': False,
 'access_level': 'owner',
 'is_owner': True,
 'entities': [{'name': 'test', 'type': 'tag'}],
 'parent_id': None,
 'meta': None}

In [27]:

try:
    deleted = await client.delete_content(content_id)
except Exception as e:
    print(e)
try:
    single_content = await client.get_content(content_id)
except Exception as e:
    print(e)
    
#OK - content unavailable after delete

Content not found or no access


In [28]:

try:
    restored = await client.restore_content(content_id)
except Exception as e:
    print(e)
try:
    single_content = await client.get_content(content_id)
except Exception as e:
    print(e)

#OK - content unavailable after delete

Content not found or not archived


In [29]:
# Add new content
content = await client.add_content(
    body="Sample content text",
    content_type="note",
    entities=[{"type": "speaker", "name": "John Doe"}]
)
content

{'content_id': 'de23ed0f-234f-4200-ae23-839e198bca15'}

In [30]:
# Sharing
share_link = await client.create_share_link(
    access_level="transcript",
    content_ids=[content_id],
    target_email="recipient@example.com",
    expiration_hours=24
)
    


In [31]:
accepted = await client.accept_share_link(
    token=share_link["token"],
    accepting_email="recipient@example.com"
)


In [32]:

# Index content
indexed = await client.index_content(content_id)
indexed



{'status': 'queued', 'message': 'Content has been queued for indexing'}

In [33]:
# Entity management
entities = await client.get_entities("speaker", offset=0, limit=20)
entities


[{'name': 'John Doe', 'type': 'speaker', 'id': 5704},
 {'name': 'Dmitriy Grankin', 'type': 'speaker', 'id': 3440},
 {'name': 'Sergey Ryabenko', 'type': 'speaker', 'id': 3441},
 {'name': 'Artem Puzik', 'type': 'speaker', 'id': 3484},
 {'name': 'Zlata Shakhovich', 'type': 'speaker', 'id': 5965}]

In [34]:
# Entity management
entities = await client.get_entities("tag", offset=0, limit=20)
entities


[{'name': 'test', 'type': 'tag', 'id': 5722}]

In [35]:
# Chat interactions
async for response in client.chat(
    query="what is vexa?",
    model="gpt-4o-mini",
    temperature=0.7
):
    print(response)



{'chunk': '**'}
{'chunk': 'V'}
{'chunk': 'exa'}
{'chunk': '**'}
{'chunk': ' is'}
{'chunk': ' a'}
{'chunk': ' term'}
{'chunk': ' that'}
{'chunk': ' can'}
{'chunk': ' refer'}
{'chunk': ' to'}
{'chunk': ' different'}
{'chunk': ' things'}
{'chunk': ' depending'}
{'chunk': ' on'}
{'chunk': ' the'}
{'chunk': ' context'}
{'chunk': '.'}
{'chunk': ' However'}
{'chunk': ','}
{'chunk': ' in'}
{'chunk': ' the'}
{'chunk': ' tech'}
{'chunk': ' and'}
{'chunk': ' business'}
{'chunk': ' landscape'}
{'chunk': ','}
{'chunk': ' it'}
{'chunk': ' is'}
{'chunk': ' often'}
{'chunk': ' associated'}
{'chunk': ' with'}
{'chunk': ' a'}
{'chunk': ' software'}
{'chunk': ' platform'}
{'chunk': ' or'}
{'chunk': ' a'}
{'chunk': ' company'}
{'chunk': ' that'}
{'chunk': ' provides'}
{'chunk': ' solutions'}
{'chunk': ' for'}
{'chunk': ' data'}
{'chunk': ' management'}
{'chunk': ','}
{'chunk': ' analytics'}
{'chunk': ','}
{'chunk': ' or'}
{'chunk': ' cloud'}
{'chunk': ' services'}
{'chunk': '.'}
{'chunk': ' \n\n'}
{'chunk

In [36]:
# Chat interactions
async for response in client.chat(
    query="who is Dmitry?",
    model="gpt-4o-mini",
    temperature=0.7
):
    print(response)



{'chunk': 'You'}
{'chunk': ' are'}
{'chunk': ' Dmit'}
{'chunk': 'riy'}
{'chunk': ' Gr'}
{'chunk': 'ank'}
{'chunk': 'in'}
{'chunk': ','}
{'chunk': ' and'}
{'chunk': ' I'}
{'chunk': ' am'}
{'chunk': ' here'}
{'chunk': ' to'}
{'chunk': ' assist'}
{'chunk': ' you'}
{'chunk': ' with'}
{'chunk': ' your'}
{'chunk': ' inquiries'}
{'chunk': ' and'}
{'chunk': ' provide'}
{'chunk': ' information'}
{'chunk': ' tailored'}
{'chunk': ' to'}
{'chunk': ' your'}
{'chunk': ' interests'}
{'chunk': ' and'}
{'chunk': ' needs'}
{'chunk': '.'}
{'chunk': ' If'}
{'chunk': ' you'}
{'chunk': ' have'}
{'chunk': ' specific'}
{'chunk': ' questions'}
{'chunk': ' or'}
{'chunk': ' topics'}
{'chunk': " you'd"}
{'chunk': ' like'}
{'chunk': ' to'}
{'chunk': ' explore'}
{'chunk': ','}
{'chunk': ' feel'}
{'chunk': ' free'}
{'chunk': ' to'}
{'chunk': ' ask'}
{'chunk': '!'}
{'thread_id': '91d1f279-13d2-4de1-8aa0-d7cd2491e676', 'content_id': None, 'entity_id': None, 'content_ids': None, 'entity_ids': None, 'output': "You are D

In [37]:
# Edit chat message
async for response in client.edit_chat_message(
    thread_id="thread_123",
    message_index=1,
    new_content="Updated message content",
    model="gpt-4o-mini"
):
    print(response)


{'error': "Edit failed: ChatManager.edit_and_continue() missing 1 required positional argument: 'context_provider'"}


In [38]:

# Thread management
threads = await client.get_threads(
 #   content_id=content_id,
    only_archived=False,
    limit=50,
    start_date=datetime.now() - timedelta(days=7)
)


In [39]:
threads

{'total': 2,
 'threads': [{'thread_id': '91d1f279-13d2-4de1-8aa0-d7cd2491e676',
   'thread_name': 'who is Dmitry?',
   'timestamp': '2025-02-04T14:59:15.640980+00:00',
   'content_id': None,
   'entity_id': None,
   'meta': None,
   'is_archived': False},
  {'thread_id': 'fb95c18b-62ec-4e92-8500-aad3567b25ef',
   'thread_name': 'what is vexa?',
   'timestamp': '2025-02-04T14:59:14.169357+00:00',
   'content_id': None,
   'entity_id': None,
   'meta': None,
   'is_archived': False}]}

In [40]:


thread = await client.get_thread("thread_123")
archived_thread = await client.archive_thread("thread_123")
restored_thread = await client.restore_thread("thread_123")
renamed = await client.rename_thread("thread_123", "New Thread Name")


ValueError: Thread not found

In [41]:

print("\nCreating user prompt...")
user_prompt = await client.create_prompt(
    prompt="What did {speaker} say about {topic}?",
    prompt_type="speaker",
    alias="speaker_topic"
)
print(f"Created user prompt with ID: {user_prompt['prompt_id']}")

print("\nGetting all prompts...")
all_prompts = await client.get_prompts()
print("All prompts:")
for p in all_prompts:
    print(f"- {p['type']}: {p['prompt']}" + (f" (alias: {p['alias']})" if p['alias'] else ""))

print("\nGetting meeting prompts...")
meeting_prompts = await client.get_prompts(prompt_type="meeting")
print("Meeting prompts:")
for p in meeting_prompts:
    print(f"- {p['prompt']}" + (f" (alias: {p['alias']})" if p['alias'] else ""))


Creating user prompt...
Created user prompt with ID: 2

Getting all prompts...
All prompts:
- speaker: What did {speaker} say about {topic}? (alias: speaker_topic)

Getting meeting prompts...
Meeting prompts:


In [7]:
print("\nCreating user prompt...")
user_prompt = await client.create_prompt(
    prompt="What did {speaker} say about {topic}?",
    prompt_type="speaker",
    alias="speaker_topic"
)
print(f"Created user prompt with ID: {user_prompt['prompt_id']}")

print("\nGetting all prompts...")
all_prompts = await client.get_prompts()
print("All prompts:")
for p in all_prompts:
    print(f"- {p['type']}: {p['prompt']}" + (f" (alias: {p['alias']})" if p['alias'] else ""))

print("\nGetting meeting prompts...")
meeting_prompts = await client.get_prompts(prompt_type="meeting")
print("Meeting prompts:")
for p in meeting_prompts:
    print(f"- {p['prompt']}" + (f" (alias: {p['alias']})" if p['alias'] else ""))

print("\nUpdating prompt...")
prompt_id = user_prompt['prompt_id']
update_result = await client.update_prompt(
    prompt_id=prompt_id,
    prompt="What did {speaker} discuss about {topic}?",
    prompt_type="speaker",
    alias="speaker_discussion"
)
print(f"Update result: {update_result}")

print("\nVerifying update...")
updated_prompts = await client.get_prompts()
for p in updated_prompts:
    if p['id'] == prompt_id:
        print(f"Updated prompt: {p['type']}: {p['prompt']}" + (f" (alias: {p['alias']})" if p['alias'] else ""))

print("\nDeleting prompt...")
delete_result = await client.delete_prompt(prompt_id)
print(f"Delete result: {delete_result}")

print("\nVerifying deletion...")
remaining_prompts = await client.get_prompts()
print(f"Remaining prompts count: {len(remaining_prompts)}")
for p in remaining_prompts:
    print(f"- {p['type']}: {p['prompt']}" + (f" (alias: {p['alias']})" if p['alias'] else ""))

# Error handling examples
print("\nTesting error cases...")
try:
    await client.update_prompt(999999, prompt="This should fail")
    print("Should not reach here")
except ValueError as e:
    print(f"Expected error on non-existent prompt: {e}")

try:
    await client.delete_prompt(999999)
    print("Should not reach here")
except ValueError as e:
    print(f"Expected error on non-existent prompt: {e}")

try:
    await client.update_prompt(prompt_id, prompt_type="invalid_type")
    print("Should not reach here")
except APIError as e:
    print(f"Expected error on invalid prompt type: {e}")


Creating user prompt...
Created user prompt with ID: 3

Getting all prompts...
All prompts:
- speaker: What did {speaker} say about {topic}? (alias: speaker_topic)

Getting meeting prompts...
Meeting prompts:

Updating prompt...
Update result: {'success': True}

Verifying update...
Updated prompt: speaker: What did {speaker} discuss about {topic}? (alias: speaker_discussion)

Deleting prompt...
Delete result: {'success': True}

Verifying deletion...
Remaining prompts count: 0

Testing error cases...
Expected error on non-existent prompt: Prompt not found
Expected error on non-existent prompt: Prompt not found


NameError: name 'APIError' is not defined